In [1]:
import ipywidgets as widgets
from IPython.display import clear_output,display

In [2]:
def upload_files(b):
    print('Files were uploaded')

In [3]:
button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button.on_click(upload_files)

display(button)

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

In [4]:
uploader = widgets.FileUpload()
display(uploader)

FileUpload(value={}, description='Upload')

In [5]:
uploader.value

{}

In [7]:
uploaded_file = uploader.value[0]
uploaded_file["size"]
uploaded_file.size

KeyError: 0

In [5]:
uploader.close()